# План домашней работы
1. Реализовать алгоритмы **item2item**, **ALS**, **IALS** (2 балл за каждый)
2. Посчитать метрику предсказаний **MRR@100** выбрасывая случайный лайк пользователя (2 балла)

Будем решать задачу предсказания: на 4/5 пользователей учимся, на 1/5 выбрасываем случайный лайк и пытаемся предсказать его беря топ 100 наших лучших предсказаний для этого пользователя.

MRR@100 будет равно $1/(p+1)$, где $p$ - позиция на которой оказался выброшенный лайк в нашем ранжировании и 0 если в топ 100 его не было.

3. Подобрать параметры алгоритмов для максимизации MRR@100 (1 балл)
4. Сравнить похожести айтемов получаюшиеся для item2item, ALS, IALS (1 балл)

Замерить насколько получаются похожими топы похожестей. Так же рекомендуется взять 5-топовых (или любимых) статей и посмотреть на похожести которые получаются для них в разных алгоритмах.

### Обозначения:
1. item_id - уникальный id айтема (статьи).
2. user_id - уникальный id пользователя.
3. source_id - уникальный id автора. Если у двух айтемов совпадают source_id, то это статьи одного автора.
4. Название айтема - это название статьи.
5. В датасете хранится user_id и список айтемов item_ids, с которыми пользователь положительно провзаимодействовал.

### Ссылки на датасет Дзена:

1. Датасет: https://disk.yandex.ru/d/uUx1MMsZUR87Sw
2. Названия айтемов: https://disk.yandex.ru/d/_ZMXsmki-OtLJA
3. Линки между item_id и source_id: https://disk.yandex.ru/d/GCryohhLbYPFoA

# Разбираемся с датасетом Дзена

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm.notebook import tqdm

In [2]:
all_names = pd.read_json("item_id_to_name.json", lines=False)
item_links = pd.read_json("item_id_to_source_id.json", lines=False)
dataset = pd.read_json("dataset_zen.json", lines=False)

In [3]:
all_names

,id,name
0,94962,Что обычно ожидало русских казачек в руках у к...
1,3972,Почему Россия решила строить новую скоростную ...
2,94644,"5 неприличных фактов об Андрее Макаревиче, кот..."
3,82518,"Что стало с красавицей Хмельницкой, которую му..."
4,53264,"Понять и Простить: Почему угонщики, бежавшие и..."
...,...,...
104498,36769,"Плюс один источник мифа о рыцарях, неспособных..."
104499,9190,Мой сад - малоуходный
104500,52731,Купил первую в жизни циркулярную пилу. Честный...
104501,72660,Решили предложить Марине помощь в лечении ч.10


In [4]:
item_links

,id,source
0,94962,2919814402697966089
1,3972,3263022753228392991
2,94644,-3857390427602554682
3,82518,-9036908390349249792
4,53264,3353856219169766284
...,...,...
104498,36769,3818746211375738614
104499,9190,4975535765688979937
104500,52731,3720366796439288909
104501,72660,-7860042973720636310


In [5]:
dataset

,user_id,item_ids
0,993675863667353526,"[15267, 61075, 81203, 17066, 25471, 88427, 638..."
1,4250619547882954185,"[4555, 94644, 84972, 17774, 94962, 78217, 2485..."
2,3847785305345691076,"[1898, 26703, 16525, 86939, 55017, 31069, 4035..."
3,1785181112918558233,"[75601, 102458, 28716, 100694, 5757, 47104, 60..."
4,5078748097863903181,"[72260, 40825, 2615, 42549, 379, 100818, 56827..."
...,...,...
75905,4954138831959898373,"[11881, 55520, 63054, 48015, 66952, 103830, 21..."
75906,4967793435819938014,"[74697, 11830, 63858, 87245, 41956, 62089, 686..."
75907,7137764184903122777,"[10353, 1775, 103680, 29704, 9782, 13295, 9975..."
75908,2624987805086334956,"[24324, 18854, 73319, 66641, 64078, 97387, 426..."


In [6]:
total_interactions_count = dataset.item_ids.map(len).sum()
user_coo = np.zeros(total_interactions_count, dtype=np.int64)
item_coo = np.zeros(total_interactions_count, dtype=np.int64)
pos = 0

for user_id, item_ids in enumerate(tqdm(dataset.item_ids)):
    user_coo[pos:pos+len(item_ids)] = user_id
    item_coo[pos:pos+len(item_ids)] = item_ids
    pos += len(item_ids)
shape = (max(user_coo) + 1, max(item_coo) + 1)
user_item_matrix = sp.coo_matrix((np.ones(len(user_coo)), (user_coo, item_coo)), shape=shape)
user_item_matrix = user_item_matrix.tocsr()
sp.save_npz("data_train.npz", user_item_matrix)
# Cleanup memory. Later you need just data_train.npz
del user_coo
del item_coo
del dataset

  0%|          | 0/75910 [00:00<?, ?it/s]

In [7]:
user_item_matrix = sp.load_npz("data_train.npz")

In [8]:
user_item_matrix

<75910x104503 sparse matrix of type '<class 'numpy.float64'>'
	with 5792423 stored elements in Compressed Sparse Row format>

In [9]:
item_weights = np.array(user_item_matrix.tocsc().sum(0))[0]
top_to_bottom_order = np.argsort(-item_weights)
item_mapping = np.empty(top_to_bottom_order.shape, dtype=int)
item_mapping[top_to_bottom_order] = np.arange(len(top_to_bottom_order))
total_item_count = (item_weights > 0).sum()
total_user_count = user_item_matrix.shape[0]

def build_dataset(user_item_matrix, item_pct, user_pct):
    user_count, item_count = int(total_user_count * user_pct), int(total_item_count * item_pct)
    item_ids = top_to_bottom_order[:item_count]
    user_ids = np.random.choice(np.arange(user_item_matrix.shape[0]), size=user_count, replace=False)
    train = user_item_matrix[user_ids]
    train = train[:, item_ids]
    return train

In [10]:
small_dataset = build_dataset(user_item_matrix, 0.05, 0.05)

In [11]:
small_dataset

<3795x5019 sparse matrix of type '<class 'numpy.float64'>'
	with 139326 stored elements in Compressed Sparse Row format>

In [113]:
def train_test_split(ratings):
    n_split = int(0.8*ratings.shape[0])
    test_split = ratings.shape[0]
#     test = np.zeros(ratings.shape)
    test = np.zeros(test_split - n_split)
    train = ratings.copy()
    for user in range(n_split, test_split):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=1, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]

        assert(np.all((train * test) == 0)) 
    return train, test

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

def item2item(dataset):
    similarities = cosine_similarity(dataset) # dense_output
    ratings = similarities @ dataset / np.sum(dataset, axis=0)
    return ratings

def item2item_tt(train, test):
    similarities = cosine_similarity(dataset) # dense_output
    dataset_new = np.concat(train, test, axis=0)   
    return (similarities @ dataset_new / np.sum(dataset_new, axis=0))[-len(test):, :]

In [15]:
rec2 = item2item(small_dataset)

In [89]:
def mrr100(predicted, actual):
    m = 0
    for i, j in zip(np.argsort(predicted, axis=1)[:, -100:], actual):
        m += 1./(100 - np.argmax(np.isin(i, j.tocoo().col))) if np.sum(np.isin(i, j.tocoo().col)) else 0
    return m / predicted.shape[0]

In [90]:
mrr100(rec2, small_dataset)

0.06163447178374308

In [161]:
from numpy.linalg import solve

class ALS():
    def __init__(self, ratings, n_factors, user_fact_reg=0.01, item_fact_reg=0.01):
        self.ratings = ratings
        self.n_users, self.n_items = ratings.shape
        self.n_factors = n_factors
        self.user_vecs = np.random.normal(scale=1./self.n_factors,\
                                          size=(self.n_users, self.n_factors))
        self.item_vecs = np.random.normal(scale=1./self.n_factors,
                                          size=(self.n_items, self.n_factors))
        self.user_fact_reg = user_fact_reg
        self.item_fact_reg = item_fact_reg

    def als_step(self, latent_vectors,
                 fixed_vecs, ratings,
                 _lambda=0.1, type='user'):
        if type == 'user':
            # Precompute
            YTY = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(YTY.shape[0]) * _lambda

            for u in range(latent_vectors.shape[0]):
                latent_vectors[u, :] = solve((YTY + lambdaI), 
                                             ratings[u, :].dot(fixed_vecs).flatten())
        elif type == 'item':
            # Precompute
            XTX = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(XTX.shape[0]) * _lambda

            for i in range(latent_vectors.shape[0]):
                latent_vectors[i, :] = solve((XTX + lambdaI), 
                                             ratings[:, i].T.dot(fixed_vecs).flatten())
        return latent_vectors

    def train(self, n_iter):
        for _ in range(n_iter):
            self.user_vecs = self.als_step(self.user_vecs, self.item_vecs, 
                                                       self.ratings, 
                                                       self.user_fact_reg, 
                                                       type='user')
            self.item_vecs = self.als_step(self.item_vecs, 
                                           self.user_vecs, 
                                           self.ratings, 
                                           self.item_fact_reg, 
                                           type='item')
            
    def predict(self, u, i):
        return self.user_vecs[u, :].dot(self.item_vecs[i, :].T)
    
    def predict_all(self):
        predictions = np.zeros((self.user_vecs.shape[0], 
                                self.item_vecs.shape[0]))
        for u in range(self.user_vecs.shape[0]):
            for i in range(self.item_vecs.shape[0]):
                predictions[u, i] = self.predict(u, i)
        return predictions

In [162]:
als = ALS(small_dataset, 10)
als.train(2)

In [163]:
preds_als = als.predict_all()

In [164]:
mrr100(preds_als, small_dataset)

0.028762330180560066

In [175]:
class IALS(ALS):
    
    def iteration(self, fixed_vecs, type='user'):
        num_solve = self.num_users if type == 'user' else self.num_items
        num_fixed = fixed_vecs.shape[0]
        YTY = fixed_vecs.T.dot(fixed_vecs)
        eye = sparse.eye(num_fixed)
        lambda_eye = self.reg_param * sparse.eye(self.num_factors)
        solve_vecs = np.zeros((num_solve, self.num_factors))

        for i in range(num_solve):
            if type=='user':
                counts_i = self.counts[i].toarray()
            else:
                counts_i = self.counts[:, i].T.toarray()
            CuI = sparse.diags(counts_i, [0])
            pu = counts_i.copy()
            pu[np.where(pu != 0)] = 1.0
            YTCuIY = fixed_vecs.T.dot(CuI).dot(fixed_vecs)
            YTCupu = fixed_vecs.T.dot(CuI + eye).dot(sparse.csr_matrix(pu).T)
            xu = spsolve(YTY + YTCuIY + lambda_eye, YTCupu)
            solve_vecs[i] = xu

        return solve_vecs

In [183]:
# item_factor, user_factor = alternating_least_squares(small_dataset, 10, 0.01)
ials = IALS(small_dataset, 10, 0.01)
ials.train(2)

In [184]:
preds_ials = ials.predict_all()

In [185]:
mrr100(preds_ials, small_dataset)

0.031058713475039617